**Genome-wide assessment identifies novel runs of homozygosity linked to Parkinson’s disease etiology across diverse ancestral populations**

---

*   **Project:** Runs of Homozygosity (ROH) Analysis in Parkinson's Disease
*   **Version:** Python/3.9
*   **Authors**: Kathryn Step, Carlos F. Hernández, Esraa Eltaraifee, Ana Jimena Hernández-Medrano, Pin-Jui Kung, Miriam Ostrožovičová, Alexandra Zirra, Zih-Hua Fang and Sara Bandres-Ciga
*   **Estimated Computation and Runtime**:
   *   **Estimated Specifications:**  1 CPU, 3.75 GB Memory, 50 GB Persistent Disk Size
   *   **Estimated Runtime:** 60 min
*   **Status:** COMPLETE
*   **Started:** 26-AUG-2023
*   **Last Updated:** 24-NOV-2024
   *   **Update Description:** Finalized ROH regions and performed downstream analysis for population-specific patterns

---

**Notebook Overview**  
*   Identifying related individuals
*   Performing QC on imputed genotype data
*   Identifying ROHs using PLINK
*   Finding overlapping ROHs in PD genes

---

**Note:**
This notebook presents the analysis for the Middle Eastern (MDE) ancestry group. The same analysis pipeline was repeated for the following additional ancestry groups:  
- **African Admixed (AAC)**  
- **African (AFR)**  
- **Ashkenazi Jewish (AJ)**  
- **American Admixed (AMR)**  
- **Central Asian (CAS)**  
- **East Asian (EAS)**  
- **European (EUR)**  
- **South Asian (SAS)**  

While only the MDE results are shown here for clarity and to minimize redundancy, the methodology and scripts are identical across all groups. Please refer to the project documentation for a comprehensive overview.  

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Create-Working-Directory" data-toc-modified-id="Create-Working-Directory-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create Working Directory</a></span></li><li><span><a href="#Download-Data" data-toc-modified-id="Download-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download Data</a></span></li><li><span><a href="#QC" data-toc-modified-id="QC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>QC</a></span><ul class="toc-item"><li><span><a href="#Identify-related-IDs-in-raw-data" data-toc-modified-id="Identify-related-IDs-in-raw-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Identify related IDs in raw data</a></span></li><li><span><a href="#Pruning-in-imputed-data" data-toc-modified-id="Pruning-in-imputed-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Pruning in imputed data</a></span></li><li><span><a href="#Merge-imputed-data" data-toc-modified-id="Merge-imputed-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Merge imputed data</a></span></li><li><span><a href="#Remove-related-individuals-in-merged-data" data-toc-modified-id="Remove-related-individuals-in-merged-data-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Remove related individuals in merged data</a></span></li></ul></li><li><span><a href="#Calculate-ROH" data-toc-modified-id="Calculate-ROH-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculate ROH</a></span></li><li><span><a href="#Intersect-PD-genes-with-ROH" data-toc-modified-id="Intersect-PD-genes-with-ROH-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Intersect PD genes with ROH</a></span><ul class="toc-item"><li><span><a href="#Download-perl-script-and-gene-list" data-toc-modified-id="Download-perl-script-and-gene-list-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Download perl script and gene list</a></span></li></ul></li><li><span><a href="#Serial-ROH" data-toc-modified-id="Serial-ROH-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Serial ROH</a></span></li></ul></div>

# Getting Started

In [1]:
# Install necessary Python packages
!pip install openpyxl --quiet

# Import required libraries
from pathlib import Path
import seaborn as sns
import openpyxl
import glob

# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

/tmp/ipykernel_1049729/298453339.py:30: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Set up R support with rpy2
!pip install --upgrade rpy2 --quiet
%reload_ext rpy2.ipython

In [3]:
%%R

# Verify that R magic is working
print('R magic cell is working')

[1] "R magic cell is working"


In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [5]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
# Utility routine for executing a shell command and returning its output
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for executing a BigQuery query
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for displaying a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''
    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)

def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))
    display_html_link(description, link_text, url)

In [6]:
%%bash

# Set up tools (PLINK, King, and ANNOVAR)
mkdir -p ~/tools
cd ~/tools

# Check and install PLINK 1.9
if test -e /home/jupyter/tools/plink; then
   echo "Plink1.9 is already installed in /home/jupyter/tools/"
else
   echo -e "Downloading plink \n    -------"
   wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
   unzip -o plink_linux_x86_64_20190304.zip
   echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "
fi

# Check and install PLINK 2.0 with a corrected download link
if test -e /home/jupyter/tools/plink2; then
   echo "Plink2 is already installed in /home/jupyter/tools/"
else
   echo -e "Downloading plink2 \n    -------"
   wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20240806.zip
   unzip -o plink2_linux_avx2_20240806.zip -d /home/jupyter/tools/
   mv /home/jupyter/tools/plink2_linux_avx2_20240806 /home/jupyter/tools/plink2
   echo -e "\nPlink2 downloaded, unzipped, and saved as 'plink2' in /home/jupyter/tools \n"
fi

# Install King 2.3.0
if test -e /home/jupyter/tools/king; then
   echo "KING is already installed in /home/jupyter/tools/"
else
   echo -e "Downloading KING \n    -------"
   wget -N https://www.kingrelatedness.com/executables/Linux-king230.tar.gz
   tar -xzvf Linux-king230.tar.gz
   echo -e "\n KING downloaded and unzipped in /home/jupyter/tools \n "
fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/
KING is already installed in /home/jupyter/tools/


In [7]:
%%bash

# List contents of the tools directory
ls /home/jupyter/tools/

king
LICENSE
Linux-king230.tar.gz
plink
plink2
plink2_linux_avx2_20240806.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


In [8]:
%%bash

# Make the downloaded tools executable
chmod u+x /home/jupyter/tools/plink
chmod u+x /home/jupyter/tools/plink2
chmod u+x /home/jupyter/tools/king

In [9]:
%%bash

# Install and set up ANNOVAR (Add download link after registration on the ANNOVAR website)
if test -e /home/jupyter/annovar; then
   echo "ANNOVAR is already installed in /home/jupyter/"
else
   echo "Downloading ANNOVAR"
   cd /home/jupyter/
   wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
   tar xvfz annovar.latest.tar.gz
fi

ANNOVAR is already installed in /home/jupyter/


In [10]:
%%capture
%%bash

# Download ANNOVAR annotation resources
cd /home/jupyter/annovar/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp150 humandb/

In [11]:
%%R

# Load necessary R packages
library(data.table)
library(magrittr)
library(stringr)
library(ggplot2)
library(plyr)
library(dplyr)
library(tidyr)

data.table 1.14.8 using 48 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘tidyr’

The following object is masked from ‘package:magrittr’:

    extract



# Create Working Directory

In [13]:
# Make a directory for use in python
print("Making a working directory")
ANC = 'MDE'
WORK_DIR = f'/home/jupyter/Release6_{ANC}'
shell_do(f'mkdir -p {WORK_DIR}')

Making a working directory


Executing: mkdir -p /home/jupyter/Release6_MDE


In [14]:
%%R

# Set directory for use in R
ANC <- 'MDE'
setwd(paste0("/home/jupyter/Release6_",ANC,"/") )

# Download Data

# QC

## Identify related IDs in raw data

In [19]:
# Extract a list of unrelated IDs in raw data
shell_do(f'/home/jupyter/tools/plink2 \
            --pfile {WORK_DIR}/{ANC}_release6 \
            --king-cutoff 0.088 \
            --out {WORK_DIR}/{ANC}_unrelated_IDs ')

Executing: /home/jupyter/tools/plink2             --pfile /home/jupyter/Release6_MDE/MDE_release6             --king-cutoff 0.088             --out /home/jupyter/Release6_MDE/MDE_unrelated_IDs 


PLINK v2.00a6LM AVX2 Intel (6 Aug 2024)        www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/MDE_unrelated_IDs.log.
Options in effect:
  --king-cutoff 0.088
  --out /home/jupyter/Release6_MDE/MDE_unrelated_IDs
  --pfile /home/jupyter/Release6_MDE/MDE_release6

Start time: Mon Oct 21 06:17:18 2024
628867 MiB RAM detected, ~614778 available; reserving 314433 MiB for main
workspace.
Using up to 96 threads (change this with --threads).
263 samples (87 females, 176 males; 263 founders) loaded from
/home/jupyter/Release6_MDE/MDE_release6.psam.
1108214 variants loaded from /home/jupyter/Release6_MDE/MDE_release6.pvar.
1 binary phenotype loaded (218 cases, 21 controls).
Excluding 37135 variants on non-autosomes from KING-robust calculation.
--king-cutoff pass 1/1: Scanning for rare variants... 121824303642485561677379859197done.
226304 variants handled by initial scan (844775 remaining).
--ki

## Pruning in imputed data

In [20]:
# Check if the combined pruned data for all chromosomes is already processed
path = Path(f'{WORK_DIR}/chrAll_{ANC}_pruned_data.bed')
if path.is_file():
    print('Files Already processed. Avoid downloading again. Showing log File.')
    shell_do(f'cat {WORK_DIR}/chr*_{ANC}_pruning.log')
    shell_do(f'cat {WORK_DIR}/chr*_{ANC}_pruned_data.log')
else:
    # Perform pruning for each chromosome (1 to 22)
    for chr in range(1, 23):
        # Check if pruning is already processed for the chromosome
        prune_path = Path(f'{WORK_DIR}/chr{chr}_{ANC}_pruning.prune.in')
        if prune_path.is_file():
            print(f'\nAlready processed chr{chr}. Showing .log file')
            shell_do(f'cat {WORK_DIR}/chr{chr}_{ANC}_pruning.log')
        else:
            # Download imputed data from cloud storage if not already done
            print(f'\nProcessing chr{chr} for pruning...')
            shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_IMPUTED_GENO_PATH}/{ANC}/chr{chr}_{ANC}_release6.* {WORK_DIR}')
            
            # Run pruning using PLINK
            shell_do(f'/home/jupyter/tools/plink2 \
                        --pfile {WORK_DIR}/chr{chr}_{ANC}_release6 \
                        --indep-pairwise 50 5 0.5 \
                        --rm-dup force-first \
                        --out {WORK_DIR}/chr{chr}_{ANC}_pruning')
        
        # Check if the pruned bed file has already been generated
        bed_path = Path(f'{WORK_DIR}/chr{chr}_{ANC}_pruned_data.bed')
        if bed_path.is_file():
            print(f'\nAlready processed chr{chr} pruned data. Showing .log file')
            shell_do(f'cat {WORK_DIR}/chr{chr}_{ANC}_pruned_data.log')
        else:
            # Generate bed file for pruned positions and filter MAF 0.05
            shell_do(f'/home/jupyter/tools/plink2 \
                        --pfile {WORK_DIR}/chr{chr}_{ANC}_release6 \
                        --extract {WORK_DIR}/chr{chr}_{ANC}_pruning.prune.in \
                        --maf 0.05 \
                        --make-bed --out {WORK_DIR}/chr{chr}_{ANC}_pruned_data')

        # Check if raw imputed files should be removed after processing
        if bed_path.is_file():
            raw_data_path = Path(f'{WORK_DIR}/chr{chr}_{ANC}_release6.pgen')
            if raw_data_path.is_file():
                print(f'Removing raw files for chr{chr}...')
                shell_do(f'rm {WORK_DIR}/chr{chr}_{ANC}_release6.p*')
            else:
                print(f"No raw files to remove for chr{chr}.")


Processing chr1 for pruning...


Executing: gsutil -u terra-1b7613a0 -m cp gs://gp2tier2/release6_21122023/imputed_genotypes/MDE/chr1_MDE_release6.* /home/jupyter/Release6_MDE


Copying gs://gp2tier2/release6_21122023/imputed_genotypes/MDE/chr1_MDE_release6.log...
Copying gs://gp2tier2/release6_21122023/imputed_genotypes/MDE/chr1_MDE_release6.pgen...
Copying gs://gp2tier2/release6_21122023/imputed_genotypes/MDE/chr1_MDE_release6.psam...
Copying gs://gp2tier2/release6_21122023/imputed_genotypes/MDE/chr1_MDE_release6.pvar...
- [4/4 files][  2.0 GiB/  2.0 GiB] 100% Done  60.8 MiB/s ETA 00:00:00           
Operation completed over 4 objects/2.0 GiB.                                      


Executing: /home/jupyter/tools/plink2                         --pfile /home/jupyter/Release6_MDE/chr1_MDE_release6                         --indep-pairwise 50 5 0.5                         --rm-dup force-first                         --out /home/jupyter/Release6_MDE/chr1_MDE_pruning


PLINK v2.00a6LM AVX2 Intel (6 Aug 2024)        www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/chr1_MDE_pruning.log.
Options in effect:
  --indep-pairwise 50 5 0.5
  --out /home/jupyter/Release6_MDE/chr1_MDE_pruning
  --pfile /home/jupyter/Release6_MDE/chr1_MDE_release6
  --rm-dup force-first

Start time: Mon Oct 21 06:17:43 2024
628867 MiB RAM detected, ~613974 available; reserving 314433 MiB for main
workspace.
Using up to 96 threads (change this with --threads).
263 samples (87 females, 176 males; 263 founders) loaded from
/home/jupyter/Release6_MDE/chr1_MDE_release6.psam.
20222554 variants loaded from
/home/jupyter/Release6_MDE/chr1_MDE_release6.pvar.
1 binary phenotype loaded (218 cases, 21 controls).
Note: Skipping --rm-dup since no duplicate IDs are present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535

...

Executing: /home/jupyter/tools/plink2                         --pfile /home/jupyter/Release6_MDE/chr22_MDE_release6                         --indep-pairwise 50 5 0.5                         --rm-dup force-first                         --out /home/jupyter/Release6_MDE/chr22_MDE_pruning


PLINK v2.00a6LM AVX2 Intel (6 Aug 2024)        www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/chr22_MDE_pruning.log.
Options in effect:
  --indep-pairwise 50 5 0.5
  --out /home/jupyter/Release6_MDE/chr22_MDE_pruning
  --pfile /home/jupyter/Release6_MDE/chr22_MDE_release6
  --rm-dup force-first

Start time: Mon Oct 21 06:27:55 2024
628867 MiB RAM detected, ~615032 available; reserving 314433 MiB for main
workspace.
Using up to 96 threads (change this with --threads).
263 samples (87 females, 176 males; 263 founders) loaded from
/home/jupyter/Release6_MDE/chr22_MDE_release6.psam.
3297419 variants loaded from
/home/jupyter/Release6_MDE/chr22_MDE_release6.pvar.
1 binary phenotype loaded (218 cases, 21 controls).
Note: Skipping --rm-dup since no duplicate IDs are present.
Calculating allele frequencies... 1113151719212325272931333537394143454749515355575961636567697173757779818385878991939

Executing: /home/jupyter/tools/plink2                         --pfile /home/jupyter/Release6_MDE/chr22_MDE_release6                         --extract /home/jupyter/Release6_MDE/chr22_MDE_pruning.prune.in                         --maf 0.05                         --make-bed --out /home/jupyter/Release6_MDE/chr22_MDE_pruned_data


PLINK v2.00a6LM AVX2 Intel (6 Aug 2024)        www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/chr22_MDE_pruned_data.log.
Options in effect:
  --extract /home/jupyter/Release6_MDE/chr22_MDE_pruning.prune.in
  --maf 0.05
  --make-bed
  --out /home/jupyter/Release6_MDE/chr22_MDE_pruned_data
  --pfile /home/jupyter/Release6_MDE/chr22_MDE_release6

Start time: Mon Oct 21 06:27:57 2024
628867 MiB RAM detected, ~615484 available; reserving 314433 MiB for main
workspace.
Using up to 96 threads (change this with --threads).
263 samples (87 females, 176 males; 263 founders) loaded from
/home/jupyter/Release6_MDE/chr22_MDE_release6.psam.
3297419 variants loaded from
/home/jupyter/Release6_MDE/chr22_MDE_release6.pvar.
1 binary phenotype loaded (218 cases, 21 controls).
--extract: 190017 variants remaining.
Calculating allele frequencies... 1012131517202224262830323335373942444649515355575961636466

Executing: rm /home/jupyter/Release6_MDE/chr22_MDE_release6.p*


## Merge imputed data

In [21]:
# Generate a list of .bed files (from chromosomes 2 to 22) to use as a merge input
shell_do(f' ls {WORK_DIR}/*_{ANC}_pruned_data.bed | sed \'s/.bed//g\' | grep -v \'chr1_\' > {WORK_DIR}/merge_list.txt ')

Executing:  ls /home/jupyter/Release6_MDE/*_MDE_pruned_data.bed | sed 's/.bed//g' | grep -v 'chr1_' > /home/jupyter/Release6_MDE/merge_list.txt 


In [22]:
# Merge pruned imputed data from all chromosomes
shell_do(f'/home/jupyter/tools/plink \
        --bfile {WORK_DIR}/chr1_{ANC}_pruned_data \
        --merge-list {WORK_DIR}/merge_list.txt \
        --make-bed --out {WORK_DIR}/chrAll_{ANC}_pruned_data ')

Executing: /home/jupyter/tools/plink         --bfile /home/jupyter/Release6_MDE/chr1_MDE_pruned_data         --merge-list /home/jupyter/Release6_MDE/merge_list.txt         --make-bed --out /home/jupyter/Release6_MDE/chrAll_MDE_pruned_data 


PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/chrAll_MDE_pruned_data.log.
Options in effect:
  --bfile /home/jupyter/Release6_MDE/chr1_MDE_pruned_data
  --make-bed
  --merge-list /home/jupyter/Release6_MDE/merge_list.txt
  --out /home/jupyter/Release6_MDE/chrAll_MDE_pruned_data

628867 MB RAM detected; reserving 314433 MB for main workspace.
to length-80+ variant IDs; consider using a different naming scheme for long
indels and the like.
position.
position.
position.
834 more same-position warnings: see log file.
Performing single-pass merge (263 people, 3989783 variants).
Merged fileset written to                     
/home/jupyter/Release6_MDE/chrAll_MDE_pruned_data-merge.bed +
/home/jupyter/Release6_MDE/chrAll_MDE_pruned_data-merge.bim +
/home/jupyter/Release6_MDE/chrAll_MDE_pruned_data-merge.fam .
3989783 variants loaded from .bim fil

## Remove related individuals in merged data

In [23]:
# Remove related IDs in merged imputed data
shell_do(f'/home/jupyter/tools/plink2 \
        --bfile {WORK_DIR}/chrAll_{ANC}_pruned_data \
        --keep {WORK_DIR}/{ANC}_unrelated_IDs.king.cutoff.in.id \
        --make-bed --out {WORK_DIR}/chrAll_{ANC}_QC ')

Executing: /home/jupyter/tools/plink2         --bfile /home/jupyter/Release6_MDE/chrAll_MDE_pruned_data         --keep /home/jupyter/Release6_MDE/MDE_unrelated_IDs.king.cutoff.in.id         --make-bed --out /home/jupyter/Release6_MDE/chrAll_MDE_QC 


PLINK v2.00a6LM AVX2 Intel (6 Aug 2024)        www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/chrAll_MDE_QC.log.
Options in effect:
  --bfile /home/jupyter/Release6_MDE/chrAll_MDE_pruned_data
  --keep /home/jupyter/Release6_MDE/MDE_unrelated_IDs.king.cutoff.in.id
  --make-bed
  --out /home/jupyter/Release6_MDE/chrAll_MDE_QC

Start time: Mon Oct 21 06:28:26 2024
628867 MiB RAM detected, ~615378 available; reserving 314433 MiB for main
workspace.
Using up to 96 threads (change this with --threads).
263 samples (87 females, 176 males; 263 founders) loaded from
/home/jupyter/Release6_MDE/chrAll_MDE_pruned_data.fam.
3989783 variants loaded from
/home/jupyter/Release6_MDE/chrAll_MDE_pruned_data.bim.
1 binary phenotype loaded (218 cases, 21 controls).
--keep: 257 samples remaining.
257 samples (85 females, 172 males; 257 founders) remaining after main filters.
214 cases and 20 controls remain

# Calculate ROH

In [31]:
# Calculate Runs of Homozygosity (ROH) using the parameters as in Table 3
shell_do(f'/home/jupyter/tools/plink \
        --bfile {WORK_DIR}/chrAll_{ANC}_QC  \
        --homozyg group --homozyg-density 50 --homozyg-gap 1000 --homozyg-kb 1500 --homozyg-snp 100 \
        --homozyg-window-het 1 --homozyg-window-missing 5 --homozyg-window-snp 50  --homozyg-window-threshold 0.05 \
        --out {WORK_DIR}/{ANC}_ROH')    

Executing: /home/jupyter/tools/plink         --bfile /home/jupyter/Release6_MDE/chrAll_MDE_QC          --homozyg group --homozyg-density 50 --homozyg-gap 1000 --homozyg-kb 1500 --homozyg-snp 100         --homozyg-window-het 1 --homozyg-window-missing 5 --homozyg-window-snp 50  --homozyg-window-threshold 0.05         --out /home/jupyter/Release6_MDE/MDE_ROH


PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/Release6_MDE/MDE_ROH.log.
Options in effect:
  --bfile /home/jupyter/Release6_MDE/chrAll_MDE_QC
  --homozyg group
  --homozyg-density 50
  --homozyg-gap 1000
  --homozyg-kb 1500
  --homozyg-snp 100
  --homozyg-window-het 1
  --homozyg-window-missing 5
  --homozyg-window-snp 50
  --homozyg-window-threshold 0.05
  --out /home/jupyter/Release6_MDE/MDE_ROH

628867 MB RAM detected; reserving 314433 MB for main workspace.
3989783 variants loaded from .bim file.
257 people (172 males, 85 females) loaded from .fam.
234 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 257 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162

# Intersect PD genes with ROH

## Download perl script and gene list

In [104]:
# Extract the first two columns from GeneList_updated.txt and remove the header
shell_do(f'cat GeneList_updated.txt | cut -f1-2 | sed 1d > geneList.txt')

Executing: cat GeneList_updated.txt | cut -f1-2 | sed 1d > geneList.txt


In [105]:
%%bash -s "$WORK_DIR" "$ANC"

WORK_DIR=$1
ANC=$2

# Navigate to the working directory
cd $WORK_DIR

# Filter and clean ROH overlap files
cat ${ANC}_ROH.hom.overlap | grep 'CON' | awk -F' ' '{if($4 != "-9")print}' > ${ANC}_ROH.hom.overlap.CON.txt
cat ${ANC}_ROH.hom.overlap | grep -v 'CON' | grep -v 'UNION' | awk -F' ' '{if($4 != "-9")print}' | grep -v 'NA' > ${ANC}_ROH.hom.overlap.clean.txt

# Run the Perl script
perl post_plink_ROH_mapping.pl ${ANC}_ROH.hom.overlap.clean.txt ${ANC}_ROH.hom.overlap.CON.txt geneList.txt ${ANC}_ROH.hom.overlap.CON.PDgenes.txt

PD gene: LOC100131289
PD gene: RSL24D1P1
PD gene: GS1-124K5.11
PD gene: AC006001.3
PD gene: IP6K2
PD gene: LRRK2
PD gene: IP6K2
PD gene: IP6K2
PD gene: SCARB2
PD gene: FAM47E
PD gene: FAM47E-STBD1
PD gene: FAM47E
PD gene: TRIM40
PD gene: SLC44A4
PD gene: IP6K2
PD gene: IP6K2
PD gene: TMEM229B
PD gene: MAP3K14
PD gene: CRHR1
PD gene: MAPT-AS1
PD gene: KANSL1
PD gene: MAPT
PD gene: MAPT
PD gene: NSF
PD gene: WNT3
PD gene: WNT3
PD gene: CHMP2B
PD gene: MTF2
PD gene: IP6K2
PD gene: SPTSSB
PD gene: SPTSSB
PD gene: SPTSSB
PD gene: SPTSSB
PD gene: SPTSSB
PD gene: SCARB2
PD gene: FAM47E
PD gene: FAM47E-STBD1
PD gene: SCARB2
PD gene: FAM47E
PD gene: FAM47E
PD gene: FGD4
PD gene: PKP2
PD gene: ALG10
PD gene: SPG11
PD gene: VAMP4
PD gene: SPTSSB
PD gene: ANK2
PD gene: CAMK2D
PD gene: CAMK2D
PD gene: CAMK2D
PD gene: CAMK2D
PD gene: ZNF608
PD gene: SYBU
PD gene: MTF2
PD gene: PMVK
PD gene: KRTCAP2
PD gene: GBAP1
PD gene: ASH1L
PD gene: PMVK
PD gene: KRTCAP2
PD gene: GBA
PD gene: GBA1
PD gene: KPNA1

In [106]:
%%R

# Load the ROH file with PD genes overlap
ROH <- fread(paste0(ANC, "_ROH.hom.overlap.CON.PDgenes.txt"), header = TRUE)

# Display the structure of the data
str(ROH)

Classes ‘data.table’ and 'data.frame':	118 obs. of  14 variables:
 $ POOL           : chr  "S10" "S1003" "S1029" "S1034" ...
 $ num_subjects   : int  21 2 2 2 2 2 2 2 2 7 ...
 $ case_to_control: chr  "18:3" "1:1" "2:0" "2:0" ...
 $ ratio          : chr  "6.0" "1.0" "2:0" "2:0" ...
 $ CHR            : chr  "chr7" "chr8" "chr11" "chr12" ...
 $ SNP1           : chr  "chr7:65881717:C:T" "chr8:16550136:C:T" "chr11:131972849:C:G" "chr12:31643136:G:A" ...
 $ SNP2           : chr  "chr7:66295242:C:G" "chr8:16779320:T:C" "chr11:133169304:T:C" "chr12:31777700:C:T" ...
 $ BP1            : int  65881717 16550136 131972849 31643136 47130562 18021081 6468460 16464732 61729448 92811215 ...
 $ BP2            : int  66295242 16779320 133169304 31777700 48766265 19237131 8035910 16580863 62746467 93055493 ...
 $ KB             : num  414 229 1196 135 1636 ...
 $ NSNP           : int  580 408 2090 212 2557 777 2305 133 1056 248 ...
 $ PD gene        : chr  "GS1-124K5.11 AC006001.3" "FGF20 AC011586.2" "IG

In [107]:
%%R

# Split the 'case_to_control' column into case and control values
ROH$case <- ldply(strsplit(as.character(ROH$case_to_control), split = ":"))[[1]]
ROH$control <- ldply(strsplit(as.character(ROH$case_to_control), split = ":"))[[2]]

# Convert case and control counts to numeric
ROH$NcasesWithROHs <- as.numeric(ROH$case)
ROH$NcontrolsWithROHs <- as.numeric(ROH$control)

# Create combined case and control columns for easy access
ROH$caseN <- ROH$NcasesWithROHs
ROH$controlN <- ROH$NcontrolsWithROHs
ROH$combinedN <- ROH$caseN + ROH$controlN

# Initialize a column for p-values
ROH$P <- NA

# Loop through the rows and perform a proportion test for each row
for (i in 1:nrow(ROH)) {
  # Perform a proportion test (note: 218 total cases, 21 total controls)
  thisP <- prop.test(x = c(ROH$NcasesWithROHs[i], ROH$NcontrolsWithROHs[i]), 
                     n = c(218, 21)) # Note: the last part should be the total N for cases and then the total N for controls
  
  # Store the p-value in the P column
  ROH$P[i] <- thisP$p.value
}

# Add a total ROH count column (sum of case and control counts)
ROH$total_ROH_count <- ROH$caseN + ROH$controlN

In addition: There were 50 or more warnings (use warnings() to see the first 50)


In [108]:
%%R

# Calculate proportions of cases and controls with ROH
ROH$propCases <- ROH$caseN / 218  # Update number of cases
ROH$propControls <- ROH$controlN / 21  # Update number of controls

# Determine if the ROH is enriched in cases
ROH$caseEnriched <- ifelse(ROH$propCases > ROH$propControls, 1, 0)

# Subset the data for ROHs that are case-enriched and have at least one ROH
ROH_subsetted <- subset(ROH, total_ROH_count >= 1 & caseEnriched == 1)

# Perform Bonferroni correction for multiple testing
Ngenes <- length(ROH_subsetted$PD)  # Number of genes in the subset
ROH_subsetted$passMultiTest <- ifelse(ROH_subsetted$P <= (0.05 / Ngenes), 1, 0)

# Calculate the Bonferroni-adjusted p-values
ROH_subsetted$BONF <- ROH_subsetted$P / Ngenes
ROH$BONF <- ROH$P / Ngenes  # Apply to the full dataset

# Subset for those that pass the multiple testing correction
significant_ROH <- subset(ROH_subsetted, passMultiTest == 1)

# Summary of how many regions pass the multiple testing correction
summary(ROH_subsetted$passMultiTest)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      0       0       0       0       0       0 


In [109]:
%%R

# Write the subsetted ROH data (with case enrichment and multiple testing) to a file
fwrite(ROH_subsetted, paste0(ANC, "_ROH_subsetted.txt"))

# Create and write the case-enriched ROH data to another file
#ROH_caseEnriched <- subset(ROH, caseEnriched == 1)
fwrite(ROH_subsetted, paste0(ANC, "_ROH_caseEnriched.txt"))

In [110]:
# Load the case-enriched ROH data using an f-string to include the ANC variable
ROH = pd.read_csv(f"{ANC}_ROH_caseEnriched.txt")

# Display the contents of the loaded dataframe
ROH

,POOL,num_subjects,case_to_control,ratio,CHR,SNP1,SNP2,BP1,BP2,KB,...,caseN,controlN,combinedN,P,total_ROH_count,propCases,propControls,caseEnriched,passMultiTest,BONF
0,S1029,2,2:0,2:0,chr11,chr11:131972849:C:G,chr11:133169304:T:C,131972849,133169304,1196.456,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
1,S1034,2,2:0,2:0,chr12,chr12:31643136:G:A,chr12:31777700:C:T,31643136,31777700,134.565,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
2,S1044,2,2:0,2:0,chr13,chr13:47130562:A:T,chr13:48766265:C:T,47130562,48766265,1635.704,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
3,S1059,2,2:0,2:0,chr17,chr17:6468460:T:A,chr17:8035910:T:A,6468460,8035910,1567.451,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
4,S1060,2,2:0,2:0,chr17,chr17:16464732:A:G,chr17:16580863:G:A,16464732,16580863,116.132,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,S896,2,2:0,2:0,chr1,chr1:231149900:A:G,chr1:232264365:T:C,231149900,232264365,1114.466,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
59,S901,2,2:0,2:0,chr2,chr2:16367931:C:T,chr2:17548048:C:T,16367931,17548048,1180.118,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
60,S902,2,2:0,2:0,chr2,chr2:18559483:A:G,chr2:19251926:C:G,18559483,19251926,692.444,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873
61,S936,2,2:0,2:0,chr3,chr3:150779749:A:G,chr3:151096608:G:C,150779749,151096608,316.860,...,2,0,2,1.0,2,0.009174,0,1,0,0.015873


In [111]:
# Filter the ROH dataframe to display only rows where the p-value (P) is <= 0.05
significant_roh = ROH[ROH['P'] <= 0.05]

# Display the filtered dataframe
significant_roh

,POOL,num_subjects,case_to_control,ratio,CHR,SNP1,SNP2,BP1,BP2,KB,...,caseN,controlN,combinedN,P,total_ROH_count,propCases,propControls,caseEnriched,passMultiTest,BONF


In [112]:
%%bash -s "$WORK_DIR" "$ANC"

WORK_DIR=$1
ANC=$2

# Navigate to the working directory
cd $WORK_DIR

{
    # Count the number of ROH overlapping PD genes, excluding the header
    printf "Number of ROH overlapping PD genes: "
    cat ${ANC}_ROH.hom.overlap.CON.PDgenes.txt | sed 1d | wc -l 

    # Count the number of ROH enriched in cases, excluding the header
    printf "Number of ROH enriched in Cases: "
    cat ${ANC}_ROH_caseEnriched.txt | sed 1d | wc -l 

    # Count the number of ROH that pass the Bonferroni correction
    printf "Number of ROH that Pass Bonferroni: "
    cat ${ANC}_ROH_caseEnriched.txt | cut -d',' -f27 | sed 1d | { grep -c '1' || true ; }
}

Number of ROH overlapping PD genes: 118
Number of ROH enriched in Cases: 63
Number of ROH that Pass Bonferroni: 0


# Serial ROH

In [ ]:
# Calculate ROH for all kb thresholds specified in the VALUES list
VALUES= [2, 3, 4, 5, 6, 7, 8, 9, 10]

for kb in VALUES:
    shell_do(f'/home/jupyter/tools/plink \
        --bfile {WORK_DIR}/chrAll_{ANC}_QC  \
        --homozyg group --homozyg-density 50 --homozyg-gap 1000 --homozyg-kb {kb}000 --homozyg-snp 100 \
        --homozyg-window-het 1 --homozyg-window-missing 5 --homozyg-window-snp 50  --homozyg-window-threshold 0.05 \
        --out {WORK_DIR}/{ANC}_ROH_{kb}kb')